In [ ]:
!pip install openai
!pip install tqdm
!pip install requests

In [ ]:
import openai
import os
import json
import base64
import requests
from tqdm import tqdm

In [ ]:
# API Schlüssel
api_key = 'sk-...'

In [ ]:
# Funktion zum Kodieren des Bildes in Base64
# Konvertiert das Bild in einen Base64-String

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Pfad zu deinem Bildverzeichnis
images_directory = "   " # Bildverzeichnis angeben

# Pfad zum Speichern der Antwort
response_save_path = "   " # Pfad angeben, wo Ergebnisse gespeichert werden

# Headers für die OpenAI API-Anfrage
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}" # API-Schlüssel für die Authentifizierung
}

# Liste, um die Ergebnisse zu speichern
results = []  # Ergebnisse der API-Anfragen hier speichern

# Alle Bilder im Verzeichnis holen
image_files = [f for f in os.listdir(images_directory) if f.endswith(".webp")]

# Fortschrittsbalken anzeigen während der Verarbeitung
for image_file in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(images_directory, image_file)
    base64_image = encode_image(image_path) # Bild kodieren
    
    # Anfrage-Payload für OpenAI API
    payload = {
        "model": "gpt-4o", # Modellname
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Count the animals in the picture. If there are any, hand them out: Yes, [number]. If there are none, enter: No, 0."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}", # Base64-kodiertes Bild
                            "detail": "low" # Detailgrad des Bildes (low oder high)
                        }
                    }
                ]
            }
        ],
        "max_tokens": 10  # Maximal zulässige Ausgabezeichen
    }
    
    # Anfrage an die OpenAI API senden
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    # Nur relevante Informationen aus der Antwort extrahieren
    result = {
        "image_file": image_file,  # Name der Bilddatei
        "response": response.json().get('choices', [{}])[0].get('message', {}).get('content', '') # Antwortinhalt
    }
    results.append(result) # Ergebnis der Liste hinzufügen

# Sicherstellen, dass das Verzeichnis existiert, in dem die Ergebnisse gespeichert werden
os.makedirs(os.path.dirname(response_save_path), exist_ok=True)

# Ergebnisse in einer JSON-Datei speichern
with open(response_save_path, 'w') as json_file:
    json.dump(results, json_file, indent=4) # Ergebnisse als JSON speichern

# Ausgabe, dass die Ergebnisse gespeichert wurden
print("Antworten wurden hier gespeichert:", response_save_path)
